In [38]:
import numpy as np
import pandas as pd
import cv2
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
import random
from shutil import copyfile

# Face Mask Detector Using Tensorflow and OpenCV/

### Number of With mask vs Without Mask

In [10]:
print("With Mask: ",len(os.listdir('Images/with_mask')))
print("Without Mask: ",len(os.listdir('Images/without_mask')))

With Mask:  690
Without Mask:  686


### Creating Train and Test Data

In [26]:
def split_data(SOURCE, TRAIN, TEST, SPLIT_SIZE):
    proper_data = []
    for img in os.listdir(SOURCE):
        img_path = SOURCE + img
        if(os.path.getsize(img_path)>0):
            proper_data.append(img)
        else:
            print("Issue in the Image:"+ img_path)
            
    training_size = int(len(proper_data)*SPLIT_SIZE)
    shuffled_data = random.sample(proper_data, len(proper_data))
    training_data = shuffled_data[:training_size]
    testing_data = shuffled_data[training_size:]
    #print(training_data)
    for img in training_data:
        temp_img = SOURCE + img
        train_img = TRAIN + img
        copyfile(temp_img, train_img)

    for img in testing_data:
        temp_img = SOURCE + img
        test_img = TEST + img
        copyfile(temp_img, test_img)

In [27]:
YES_Source = 'Images/with_mask/'
TRAINING_YES_Source = 'Images/train/with_mask/'
TESTING_YES_Source = 'Images/test/with_mask/'
NO_Source = 'Images/without_mask/'
TRAINING_NO_Source = 'Images/train/without_mask/'
TESTING_NO_Source = 'Images/test/without_mask/'
split_size = 0.8
split_data(YES_Source, TRAINING_YES_Source, TESTING_YES_Source, split_size)
split_data(NO_Source, TRAINING_NO_Source, TESTING_NO_Source, split_size)

In [28]:
print('Number of training with mask: ', len(os.listdir(TRAINING_YES_Source)))
print('Number of training without mask: ', len(os.listdir(TRAINING_NO_Source)))
print('Number of testing with mask: ', len(os.listdir(TESTING_YES_Source)))
print('Number of testing without mask: ', len(os.listdir(TESTING_NO_Source)))

Number of training with mask:  552
Number of training without mask:  548
Number of testing with mask:  138
Number of testing without mask:  138


### Creating CNN Model using TensorFlow

In [29]:
#Creating the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [30]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [32]:
TRAIN_DIR = 'Images/train/'
train_datagen = ImageDataGenerator(rescale=1/255.0,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(TRAIN_DIR,
                                                    target_size = (150,150),
                                                    class_mode='categorical',
                                                    batch_size = 32)
TEST_DIR = 'Images/test/'
test_datagen = ImageDataGenerator(rescale = 1/255.0)
test_generator = test_datagen.flow_from_directory(TEST_DIR,
                                                target_size = (150,150),
                                                class_mode='categorical',
                                                batch_size = 32)

Found 1100 images belonging to 2 classes.
Found 276 images belonging to 2 classes.


In [34]:
#Used to get best weights used when monitored with validation loss
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

In [36]:
history = model.fit_generator(train_generator,
                             epochs = 30,
                             validation_data=test_generator,
                             callbacks=[checkpoint])

Epoch 1/30
35/35 [==============================] - 47s 1s/step - loss: 0.1600 - acc: 0.9464 - val_loss: 0.1062 - val_acc: 0.9783
Epoch 2/30
35/35 [==============================] - 46s 1s/step - loss: 0.2422 - acc: 0.9282 - val_loss: 0.1133 - val_acc: 0.9746
Epoch 3/30
35/35 [==============================] - 46s 1s/step - loss: 0.2013 - acc: 0.9427 - val_loss: 0.0851 - val_acc: 0.9783
Epoch 4/30
35/35 [==============================] - 48s 1s/step - loss: 0.1638 - acc: 0.9409 - val_loss: 0.0849 - val_acc: 0.9783
Epoch 5/30
35/35 [==============================] - 53s 2s/step - loss: 0.1251 - acc: 0.9527 - val_loss: 0.1486 - val_acc: 0.9674
Epoch 6/30
35/35 [==============================] - 54s 2s/step - loss: 0.1871 - acc: 0.9373 - val_loss: 0.0858 - val_acc: 0.9710
Epoch 7/30
35/35 [==============================] - 53s 2s/step - loss: 0.1368 - acc: 0.9482 - val_loss: 0.0953 - val_acc: 0.9601
Epoch 8/30
35/35 [==============================] - 55s 2s/step - loss: 0.1149 - acc: 0.96

In [54]:
'''Remove the commented Code lines to save your Video'''

labels = {0:'No Mask, Please wear it!',1:'Mask On, Good work!'}
color = {0:(0,0,255), 1:(255,0,0)}

webcam = cv2.VideoCapture(0)

classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
#fourcc = cv2.VideoWriter_fourcc(*'ADV1')
#out = cv2.VideoWriter('lol.mp4', fourcc, 25, (640,480))
while webcam.isOpened():
    _, frame = webcam.read()
    frame = cv2.flip(frame, 1, 1)
    faces = classifier.detectMultiScale(frame, 1.1, 4)
    
    for (x,y,w,h) in faces:
        #cv2.rectangle(frame, (x,y), (x+w, y+h), (255,0,0), 3)
        face_data = frame[y:y+h, x:x+w]
        resize_data = cv2.resize(face_data, (150,150))
        resize_data = resize_data/255.0
        final_data = np.expand_dims(resize_data, axis = 0)
        prediction = model.predict(final_data)
        
        binary_answer = np.argmax(prediction,axis=1)[0]
        
        cv2.rectangle(frame, (x,y), (x+w, y+h), color[binary_answer], 3)
        cv2.rectangle(frame, (x,y-40), (x+w, y), color[binary_answer], -1)
        cv2.putText(frame, labels[binary_answer], (x,y-10), cv2.FONT_HERSHEY_PLAIN, 1, (255,255,255), 1)
    
    cv2.imshow("CORONA MASK", frame)
    #out.write(frame)
        
    if cv2.waitKey(1) == 27:
            break

#out.release()
webcam.release()
cv2.destroyAllWindows()